In [ ]:
import json
import requests
import time
from tqdm import tqdm
from urllib.parse import quote

In [ ]:
API_KEY= 'awg8IsFn6Cd78OjMoOWH8Py4gySg7BnyB01qpx_3Mq4UbNKv-3LP01UkUF3jJkGzYNkMFDisY-YL60geL1iTVvdfc_NX4bFzpgkmiVvyBn9ze-9pmo5uZpmR3aEVYnYx'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'

In [ ]:
SEARCH_LIMIT = 50

In [ ]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [ ]:
def search(api_key, term, location, offset):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'offset': offset
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


In [ ]:
def query_api(term, location, offset):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location, offset)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return 'END'
    return response

In [ ]:
food_list = ['italian', 'japanese', 'american', 'chinese', 'mexican']
for food in tqdm(food_list):
    for i in range(30):
        data = query_api(f'{food} restaurants', 'united states', i*SEARCH_LIMIT)
        if data == 'END':
            break
        else:
            j_string = json.dumps(data)
            f = open(f'data/{food}/file_{i+1}.json','w+')
            f.write(j_string)
            f.close()
        time.sleep(0.5)

In [ ]:
f = open('data/italian/file_1.json', 'r')
r_data = json.load(f)
f.close()
sample = r_data['businesses'][0]
{
    'insertedAtTimestamp' : str(datetime.now()),
    'Business_ID' : sample['id'],
    'name':sample['name'],
    'address': ' '.join(sample['location']['display_address']),
    'coordinates':{'latitude': Decimal(str(sample['coordinates']['latitude'])), 'longitude': Decimal(str(sample['coordinates']['longitude']))},
    'review_count': sample['review_count'],
    'rating': Decimal(str(sample['rating'])),
    'zip_code' : sample['location']['zip_code']
}

In [ ]:
import os
import json
from datetime import datetime

In [ ]:
all_id_key = []
AAA = []
test = []

In [ ]:
food_list = ['italian', 'japanese', 'american', 'chinese', 'mexican']
for food in food_list:
    food_ids = [] = []
    for filename in os.listdir(f'data/{food}'):
        data = json.load(open(f'data/{food}/{filename}', 'r'))
        for sample in data['businesses']:
            test.append(sample['id'])
            if sample['id'] not in all_id_key:
                all_id_key.append(sample['id'])
                food_ids.append(sample['id'])
                req_json = {
                    'insertedAtTimestamp' : str(datetime.now()),
                    'Business_ID' : sample['id'],
                    'name':sample['name'],
                    'address': ' '.join(sample['location']['display_address']),
                    'coordinates':sample['coordinates'],
                    'review_count': sample['review_count'],
                    'rating': sample['rating'],
                    'zip_code' : sample['location']['zip_code']
                }
    AAA.append(food_ids)
    

In [ ]:
for l,n in zip(AAA, food_list):
    print(f'{n} : {len(l)}')

In [ ]:
len(all_id_key)

In [ ]:
1000+810+447+436+542

## Insert into dynamoDB

In [1]:
import os
import json
import boto3
from decimal import Decimal
from datetime import datetime
import time
from tqdm.notebook import tqdm

In [2]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('yelp-restaurants')

In [3]:
def put_restaurant(data, table):
    response = table.put_item(Item=data)
    return response


In [ ]:
all_id = []
count = 0
food_list = ['italian', 'japanese', 'american', 'chinese', 'mexican']
for food in tqdm(food_list):
    for filename in tqdm(os.listdir(f'data/{food}'), leave = False):
        data = json.load(open(f'data/{food}/{filename}', 'r'))
        for sample in tqdm(data['businesses'], leave = False):
            if sample['id'] not in all_id:
                all_id.append(sample['id'])
                req_json = {
                    'insertedAtTimestamp' : str(datetime.now()),
                    'business_id' : sample['id'],
                    'name':sample['name'],
                    'address': ' '.join(sample['location']['display_address']),
                    'coordinates':{'latitude': Decimal(str(sample['coordinates']['latitude'])), 'longitude': Decimal(str(sample['coordinates']['longitude']))},
                    'review_count': sample['review_count'],
                    'rating': Decimal(str(sample['rating'])),
                    'zip_code' : sample['location']['zip_code']
                }
                count += 1
                put_restaurant(req_json, table)
            time.sleep(0.1)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
count

In [ ]:
import random
a = list(range(10))
random.sample(a, k = 5)

In [2]:
import requests
host = 'https://search-dcc-7e5nm62qe5zyy3sqeu5tmwtp4q.us-east-1.es.amazonaws.com' 
path = '/restaurants/Restaurant/1/' 

url = host + path
# print(url)
payload = {"director": "Burton, Tim", "genre": ["Comedy","Sci-Fi"], "year": 1996, "actor": ["Jack Nicholson","Pierce Brosnan","Sarah Jessica Parker"], "title": "Mars Attacks!"}
r = requests.post(url, auth=("test", "Test@1234"), json=payload) 
print(r.text)

{"_index":"restaurants","_type":"Restaurant","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}


In [ ]:
url = host + '/_search?q=year:1996'
url

In [ ]:
r = requests.get(url, auth=("test", "Test@1234")) 
r.text

In [ ]:
!pip install opensearch-py

In [ ]:
import json
import boto3
# from elasticsearch import Elasticsearch, RequestsHttpConnection
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

# TABLE_NAME = 'yelp-restaurants'
SAMPLE_N = '5'
SEARCH_URL = 'https://search-test-tsguhysu7spq7iqheokd7pma7a.us-east-1.es.amazonaws.com'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, 'us-east-1', 'es',session_token=credentials.token)

# sqs = boto3.resource('sqs',region_name='us-east-1')
es = OpenSearch(SEARCH_URL, http_auth=awsauth, connection_class=RequestsHttpConnection)


In [ ]:
data = es.search(index="restaurants", body={"query": {"match": {'director':'Burton'}}})
print("search complete", data['hits']['hits'])

In [ ]:
po

In [ ]:
es = OpenSearch(SEARCH_URL, http_auth=awsauth, connection_class=RequestsHttpConnection)

In [ ]:
payload

In [ ]:
!pip install requests-aws4auth

In [ ]:
!pip install boto3

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
for i in tqdm(range(1000)):
    for j in tqdm(range(100), leave = False):
        time.sleep(0.1)